In [1]:
# -*- coding: utf-8 -*-

# === KHÔNG THAY ĐỔI (Imports cơ bản và thêm thư viện cho HTR) ===
import os


import pandas as pd
import numpy as np
# import sns # Không cần thiết cho HTR cơ bản
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt # Thêm để vẽ đồ thị loss (tùy chọn)

In [2]:
# === KHÔNG THAY ĐỔI (Thiết lập đường dẫn) ===
# Đường dẫn thư mục dữ liệu
data_dir = r"E:\HANDS-VNOnDB" # <--- KIỂM TRA LẠI ĐƯỜNG DẪN NÀY
label_file_path = os.path.join(data_dir, "normalized_InkData_line.csv") # <--- File chứa cả ID và label text
image_folder_original = os.path.join(data_dir, "InkData_line")

# Thư mục lưu ảnh đã xử lý (resize)
image_folder_processed = r"D:\HANDS-VNOnDB\Processed_Images_Lines_HTR" 

In [3]:
# 3. Chuẩn hóa file gán nhãn

# def split_id(df, id_col="id"):
#     df[['document_id', 'line_id', 'word_id']] = df[id_col].str.rsplit('_', n=2, expand=True)
#     return df.head()

print((pd.read_csv(os.path.join(data_dir, "normalized_InkData_line.csv"))))

                                id  \
0       20140603_0003_BCCTC_tg_0_0   
1       20140603_0003_BCCTC_tg_0_1   
2       20140603_0003_BCCTC_tg_0_2   
3       20140603_0003_BCCTC_tg_0_3   
4       20140603_0003_BCCTC_tg_0_4   
...                            ...   
7291  20160722_0202_26749_1_tg_6_8   
7292  20160722_0202_26749_1_tg_7_0   
7293  20160722_0202_26749_1_tg_7_1   
7294  20160722_0202_26749_1_tg_7_2   
7295  20160722_0202_26749_1_tg_7_3   

                                                  label  \
0                               Bản chất của thành công   
1     Đã bao giờ bạn tự hỏi thành công là gì mà bao ...   
2     chăng đó là kết quả hoàn hảo trong công việc, ...   
3     là cách nói khác của từ thành đạt, nghĩa là có...   
4     mọi người nể phục? Vậy thì bạn hãy dành chút t...   
...                                                 ...   
7291        nói thì bao giờ tôi cũng nói có lợi về mình   
7292  Các phóng viên đã đặt khá nhiều câu hỏi, nhưng...   
7293  từ chố

In [4]:
TARGET_IMG_HEIGHT = 32
TARGET_IMG_WIDTH = 128

In [5]:
def preprocess_image_htr(image_path, target_size=(TARGET_IMG_WIDTH, TARGET_IMG_HEIGHT)):
    """Mở ảnh, chuyển grayscale, resize."""
    try:
        with Image.open(image_path) as img:
            img_gray = img.convert('L')
            img_resized = img_gray.resize(target_size, Image.Resampling.LANCZOS)
        return img_resized
    except Exception as e:
        print(f"Lỗi khi mở hoặc xử lý ảnh {image_path}: {e}")
        return None

In [6]:
def process_and_save_images_htr(input_folder, output_folder, target_size=(TARGET_IMG_WIDTH, TARGET_IMG_HEIGHT)):
    """Tiền xử lý và lưu ảnh vào thư mục mới."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Đã tạo thư mục: {output_folder}")

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg'))]
    print(f"Tìm thấy {len(image_files)} ảnh trong {input_folder}")

    processed_count = 0
    skipped_count = 0
    error_count = 0
    for img_file in tqdm(image_files, desc="Preprocessing images for HTR", unit="image"):
        img_path = os.path.join(input_folder, img_file)
        output_img_path = os.path.join(output_folder, img_file)

        if not os.path.exists(output_img_path):
            processed_img = preprocess_image_htr(img_path, target_size)
            if processed_img:
                try:
                    processed_img.save(output_img_path)
                    processed_count += 1
                except Exception as e:
                    print(f"Lỗi khi lưu ảnh {output_img_path}: {e}")
                    error_count += 1
            else:
                 error_count += 1 # Lỗi trong hàm preprocess_image_htr
        else:
            skipped_count += 1

    total_processed_or_skipped = processed_count + skipped_count
    print(f"Hoàn thành tiền xử lý:")
    print(f"- Đã xử lý mới: {processed_count}")
    print(f"- Bỏ qua (đã tồn tại): {skipped_count}")
    print(f"- Lỗi: {error_count}")
    print(f"- Tổng số ảnh trong thư mục output: {total_processed_or_skipped}")
    print(f"Ảnh đã xử lý được lưu tại: {output_folder}")

In [7]:
# Chạy tiền xử lý ảnh (chỉ chạy nếu thư mục processed chưa có hoặc muốn ghi đè)
run_preprocessing = True # Đặt thành False nếu đã chạy và không muốn chạy lại
if run_preprocessing:
    process_and_save_images_htr(image_folder_original, image_folder_processed, target_size=(TARGET_IMG_WIDTH, TARGET_IMG_HEIGHT))
else:
    print(f"Bỏ qua bước tiền xử lý ảnh, sử dụng ảnh tại: {image_folder_processed}")
    if not os.path.exists(image_folder_processed):
        print(f"CẢNH BÁO: Thư mục ảnh đã xử lý {image_folder_processed} không tồn tại nhưng run_preprocessing=False.")
        exit()


Tìm thấy 7296 ảnh trong E:\HANDS-VNOnDB\InkData_line


Preprocessing images for HTR: 100%|██████████| 7296/7296 [00:00<00:00, 13059.01image/s]

Hoàn thành tiền xử lý:
- Đã xử lý mới: 0
- Bỏ qua (đã tồn tại): 7296
- Lỗi: 0
- Tổng số ảnh trong thư mục output: 7296
Ảnh đã xử lý được lưu tại: D:\HANDS-VNOnDB\Processed_Images_Lines_HTR


In [8]:
# === THAY ĐỔI (Load nhãn TEXT thay vì ID) ===
# Đọc file CSV chứa nhãn
try:
    labels_df = pd.read_csv(label_file_path)
    print(f"Đã đọc thành công file label: {label_file_path}")
    # print("5 dòng đầu tiên của labels_df:")
    # print(labels_df.head())
    if 'label' not in labels_df.columns or 'id' not in labels_df.columns:
        raise ValueError("File CSV phải chứa cột 'id' và 'label'.")
    # Xử lý giá trị NaN trong cột label (thay bằng chuỗi rỗng hoặc bỏ qua)
    labels_df['label'] = labels_df['label'].fillna('')
    print(f"Số lượng dòng trong file label: {len(labels_df)}")
except Exception as e:
    print(f"LỖI khi đọc hoặc kiểm tra file CSV: {e}")
    exit()

Đã đọc thành công file label: E:\HANDS-VNOnDB\normalized_InkData_line.csv
Số lượng dòng trong file label: 7296


In [9]:

# Hàm tải đường dẫn ảnh và nhãn TEXT, đảm bảo khớp ID và ảnh tồn tại
def load_image_paths_and_text_labels(processed_image_folder, labels_df):
    image_paths = []
    text_labels = []
    missing_images = 0
    valid_ids_from_df = set(labels_df['id'])
    processed_image_files = os.listdir(processed_image_folder)
    processed_ids = set(os.path.splitext(f)[0] for f in processed_image_files if f.endswith(('.png', '.jpg')))

    print(f"Số ID hợp lệ từ CSV: {len(valid_ids_from_df)}")
    print(f"Số ảnh tìm thấy trong thư mục đã xử lý: {len(processed_ids)}")

    # Lọc DataFrame để chỉ giữ lại các ID có ảnh tương ứng
    labels_df_filtered = labels_df[labels_df['id'].isin(processed_ids)].copy()
    print(f"Số lượng mẫu sau khi lọc (có cả ảnh và label): {len(labels_df_filtered)}")

    # Tạo dictionary từ ID sang label text (từ DataFrame đã lọc)
    id_to_label = pd.Series(labels_df_filtered.label.values, index=labels_df_filtered.id).to_dict()

    # Lặp qua các ID đã lọc để tạo list cuối cùng
    for img_id in tqdm(labels_df_filtered['id'], desc="Khớp ảnh và nhãn"):
        img_filename = f"{img_id}.png" # Hoặc .jpg nếu có
        img_path = os.path.join(processed_image_folder, img_filename)
        # Kiểm tra lại lần nữa (dù đã lọc)
        if os.path.exists(img_path):
            image_paths.append(img_path)
            text_labels.append(id_to_label[img_id])
        else:
             missing_images += 1 # Không nên xảy ra nếu lọc đúng

    if missing_images > 0:
        print(f"Cảnh báo: {missing_images} ảnh có trong df_filtered nhưng không tìm thấy file!")

    return image_paths, text_labels

In [13]:
# Tải dữ liệu
all_image_paths, all_text_labels = load_image_paths_and_text_labels(image_folder_processed, labels_df)

if not all_image_paths:
    print("Lỗi: Không tải được ảnh hoặc nhãn nào. Kiểm tra lại đường dẫn và file.")
    exit()

print(f"Đã tải thành công {len(all_image_paths)} cặp ảnh-nhãn.")
print("Ví dụ 5 nhãn đầu tiên:", all_text_labels[:5])

Số ID hợp lệ từ CSV: 7296
Số ảnh tìm thấy trong thư mục đã xử lý: 7296
Số lượng mẫu sau khi lọc (có cả ảnh và label): 7296


Khớp ảnh và nhãn: 100%|██████████| 7296/7296 [00:00<00:00, 21903.80it/s]

Đã tải thành công 7296 cặp ảnh-nhãn.
Ví dụ 5 nhãn đầu tiên: ['Bản chất của thành công', 'Đã bao giờ bạn tự hỏi thành công là gì mà bao kẻ bỏ cả cuộc đời mình theo đuổi? Phải', 'chăng đó là kết quả hoàn hảo trong công việc, sự chính xác đến từng chi tiết? Hay đó', 'là cách nói khác của từ thành đạt, nghĩa là có được một cuộc sống giàu sang, được', 'mọi người nể phục? Vậy thì bạn hãy dành chút thời gian để lặng mình suy ngẫm.']


In [14]:
# === THAY ĐỔI (Chia Train/Test cho dữ liệu HTR) ===
# Đường dẫn lưu file train/test sau khi chia
train_csv_path = os.path.join(data_dir, "train_htr_lines.csv")
test_csv_path = os.path.join(data_dir, "test_htr_lines.csv")
from sklearn.model_selection import train_test_split

X_train_paths, X_test_paths, y_train_text, y_test_text = train_test_split(
    all_image_paths, all_text_labels, test_size=0.2, random_state=42
)

print(f"Số lượng mẫu train: {len(X_train_paths)}")
print(f"Số lượng mẫu test: {len(X_test_paths)}")

try:
    # Tạo DataFrame và lưu train set
    train_data_to_save = pd.DataFrame({'image_path': X_train_paths, 'label': y_train_text})
    train_data_to_save.to_csv(train_csv_path, index=False, encoding='utf-8')
    print(f"Đã lưu tập train vào: {train_csv_path}")

    # Tạo DataFrame và lưu test set
    test_data_to_save = pd.DataFrame({'image_path': X_test_paths, 'label': y_test_text})
    test_data_to_save.to_csv(test_csv_path, index=False, encoding='utf-8')
    print(f"Đã lưu tập test vào: {test_csv_path}")
except Exception as e:
    print(f"Lỗi khi lưu file train/test CSV: {e}")


Số lượng mẫu train: 5836
Số lượng mẫu test: 1460
Đã lưu tập train vào: E:\HANDS-VNOnDB\train_htr_lines.csv
Đã lưu tập test vào: E:\HANDS-VNOnDB\test_htr_lines.csv


In [15]:
# === THAY ĐỔI (Tạo bộ ký tự và mã hóa nhãn cho HTR) ===

# Tìm tất cả các ký tự duy nhất từ tập train và test
all_texts_for_vocab = y_train_text + y_test_text
characters = set(char for text in all_texts_for_vocab for char in str(text))
characters = sorted(list(characters))

# Thêm ký tự đặc biệt 'blank' cho CTC Loss (thường ở index 0)
BLANK_TOKEN = '<blank>'
if BLANK_TOKEN not in characters:
    characters.insert(0, BLANK_TOKEN) # Đảm bảo blank ở vị trí 0

print(f"Tổng số ký tự (bao gồm blank): {len(characters)}")
# print("Bộ ký tự:", "".join(characters)) # Có thể rất dài

# Tạo từ điển ánh xạ
char_to_int = {char: i for i, char in enumerate(characters)}
int_to_char = {i: char for i, char in enumerate(characters)}

# Số lượng lớp cho mô hình = số ký tự
num_classes_htr = len(characters)
print(f"Số lớp đầu ra cho mô hình HTR: {num_classes_htr}")

# Hàm mã hóa text thành chuỗi số nguyên
def encode_text(text, char_to_int_map):
    return [char_to_int_map[char] for char in str(text) if char in char_to_int_map]

# Mã hóa nhãn train và test
y_train_encoded = [encode_text(text, char_to_int) for text in y_train_text]
y_test_encoded = [encode_text(text, char_to_int) for text in y_test_text]

# In thử vài mẫu đã mã hóa
print("\nVí dụ mã hóa:")
print("Nhãn gốc:", y_train_text[0])
print("Nhãn mã hóa:", y_train_encoded[0])
print("Giải mã lại:", "".join([int_to_char[i] for i in y_train_encoded[0]]))

Tổng số ký tự (bao gồm blank): 162
Số lớp đầu ra cho mô hình HTR: 162

Ví dụ mã hóa:
Nhãn gốc: Ngược lại, không có việc thì chúng tôi về, không phải tự nguyện xin về
Nhãn mã hóa: [39, 57, 107, 150, 53, 1, 62, 108, 59, 9, 1, 61, 58, 93, 64, 57, 1, 53, 92, 1, 72, 59, 133, 53, 1, 70, 58, 89, 1, 53, 58, 96, 64, 57, 1, 70, 93, 59, 1, 72, 128, 9, 1, 61, 58, 93, 64, 57, 1, 66, 58, 110, 59, 1, 70, 158, 1, 64, 57, 71, 74, 133, 64, 1, 73, 59, 64, 1, 72, 128]
Giải mã lại: Ngược lại, không có việc thì chúng tôi về, không phải tự nguyện xin về


In [17]:
# --- THÊM CODE LƯU FILE ĐÃ MÃ HÓA ---
import json
from tqdm import tqdm # Đảm bảo tqdm đã được import
print("\nBắt đầu lưu file đã mã hóa và bản đồ ký tự...")

# Định nghĩa đường dẫn file lưu (cùng thư mục với train/test csv)
encoded_train_path = os.path.join(data_dir, "encoded_train_labels.json")
encoded_test_path = os.path.join(data_dir, "encoded_test_labels.json")
charmap_path = os.path.join(data_dir, "character_map.json")

try:
    # Lưu y_train_encoded
    with open(encoded_train_path, 'w', encoding='utf-8') as f_train:
        json.dump(y_train_encoded, f_train)
    print(f"Đã lưu nhãn train đã mã hóa vào: {encoded_train_path}")

    # Lưu y_test_encoded
    with open(encoded_test_path, 'w', encoding='utf-8') as f_test:
        json.dump(y_test_encoded, f_test)
    print(f"Đã lưu nhãn test đã mã hóa vào: {encoded_test_path}")

    # Lưu cả hai bản đồ ký tự vào một file
    # Chuyển key của int_to_char thành string để tương thích JSON
    int_to_char_str_keys = {str(k): v for k, v in int_to_char.items()}
    char_maps = {
        'char_to_int': char_to_int,
        'int_to_char': int_to_char_str_keys # Lưu phiên bản key là string
    }
    with open(charmap_path, 'w', encoding='utf-8') as f_map:
        # indent=4 để dễ đọc file JSON hơn
        # ensure_ascii=False để lưu đúng ký tự tiếng Việt
        json.dump(char_maps, f_map, ensure_ascii=False, indent=4)
    print(f"Đã lưu bản đồ ký tự vào: {charmap_path}")

except Exception as e:
    print(f"Lỗi khi lưu file JSON đã mã hóa: {e}")
# --- KẾT THÚC CODE LƯU FILE ---


Bắt đầu lưu file đã mã hóa và bản đồ ký tự...
Đã lưu nhãn train đã mã hóa vào: E:\HANDS-VNOnDB\encoded_train_labels.json
Đã lưu nhãn test đã mã hóa vào: E:\HANDS-VNOnDB\encoded_test_labels.json
Đã lưu bản đồ ký tự vào: E:\HANDS-VNOnDB\character_map.json


In [20]:
# Transforms cho ảnh HTR
htr_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), # Đảm bảo ảnh là 1 kênh màu
    transforms.ToTensor(), # Chuyển sang Tensor và chuẩn hóa về [0, 1]
    transforms.Normalize((0.5,), (0.5,)) # Chuẩn hóa về [-1, 1] (phổ biến cho HTR)
])

In [21]:
class CustomDatasetHTR(Dataset):
    def __init__(self, image_paths, encoded_labels, transform=None):
        self.image_paths = image_paths
        self.encoded_labels = encoded_labels # List các list số nguyên
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        encoded_label = self.encoded_labels[idx] # Lấy list số nguyên

        try:
            # Mở ảnh bằng Pillow trước khi transform
            image = Image.open(img_path) # Không convert ở đây, để transform xử lý
            if self.transform:
                image = self.transform(image)
            # Trả về ảnh tensor và label là tensor số nguyên
            return image, torch.tensor(encoded_label, dtype=torch.long)
        except Exception as e:
            print(f"Lỗi khi tải hoặc transform ảnh {img_path}: {e}")
            # Trả về None hoặc sample trống để collate_fn có thể xử lý
            # Hoặc có thể bỏ qua sample này trong __init__
            return None, None # Cần xử lý trong collate_fn

In [22]:
# Hàm collate_fn để xử lý batch có độ dài khác nhau
# Cần xác định chiều rộng output của CNN (cnn_output_width)
# ---- ƯỚC LƯỢNG CNN OUTPUT WIDTH ----
# Với input W=256 và MobileNetV3 Small, sau các lớp stride=2, chiều rộng có thể giảm đi đáng kể.
# VD: 256 -> 128 -> 64 -> 32 -> 16. Giả sử output là W=16 (Cần kiểm tra lại!)
# Hoặc có thể không cần stride ở chiều rộng nhiều như vậy trong HTR.
# Tạm thời ước lượng:
CNN_OUTPUT_WIDTH = 64 # <--- GIÁ TRỊ NÀY RẤT QUAN TRỌNG, CẦN KIỂM TRA LẠI SAU KHI CÓ MODEL CNN

def collate_fn_htr(batch):
    # Lọc bỏ các sample bị lỗi (None)
    batch = [item for item in batch if item[0] is not None]
    if not batch:
        return None, None, None, None # Trả về None nếu cả batch bị lỗi

    images, labels = zip(*batch)

    # Xếp chồng ảnh thành batch (N, C, H, W)
    images = torch.stack(images, 0)

    # Pad labels
    label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long)
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=char_to_int[BLANK_TOKEN]) # Pad bằng blank token

    # Tính toán input lengths cho CTCLoss (chiều dài sequence từ CNN output)
    input_lengths = torch.full(size=(len(images),), fill_value=CNN_OUTPUT_WIDTH, dtype=torch.long)

    return images, padded_labels, input_lengths, label_lengths

In [18]:
# --- Thử load dữ liệu từ file ---
print("\nKiểm tra và thử tải dữ liệu đã mã hóa từ file JSON...")
if os.path.exists(encoded_train_path) and \
   os.path.exists(encoded_test_path) and \
   os.path.exists(charmap_path):
    try:
        print(f"Đang tải nhãn train từ: {encoded_train_path}")
        with open(encoded_train_path, 'r', encoding='utf-8') as f_train:
            y_train_encoded = json.load(f_train)

        print(f"Đang tải nhãn test từ: {encoded_test_path}")
        with open(encoded_test_path, 'r', encoding='utf-8') as f_test:
            y_test_encoded = json.load(f_test)

        print(f"Đang tải bản đồ ký tự từ: {charmap_path}")
        with open(charmap_path, 'r', encoding='utf-8') as f_map:
            char_maps_loaded = json.load(f_map)
            char_to_int = char_maps_loaded['char_to_int']
            # Tái tạo int_to_char với key là integer từ char_to_int
            int_to_char = {v: k for k, v in char_to_int.items()}

        # Tính lại số lớp từ char_map đã load
        num_classes_htr = len(char_to_int) # Hoặc len(int_to_char)
        print(f"Tải thành công! Số lớp: {num_classes_htr}")
        loaded_encoded_data = True

    except Exception as e:
        print(f"Lỗi khi tải file JSON đã mã hóa: {e}. Sẽ tiến hành mã hóa lại.")
        # Reset biến về None để đảm bảo chạy mã hóa lại
        y_train_encoded = None
        y_test_encoded = None
        char_to_int = None
        int_to_char = None
        num_classes_htr = None
else:
    print("Không tìm thấy file JSON đã mã hóa. Sẽ tiến hành mã hóa lại.")



Kiểm tra và thử tải dữ liệu đã mã hóa từ file JSON...
Đang tải nhãn train từ: E:\HANDS-VNOnDB\encoded_train_labels.json
Đang tải nhãn test từ: E:\HANDS-VNOnDB\encoded_test_labels.json
Đang tải bản đồ ký tự từ: E:\HANDS-VNOnDB\character_map.json
Tải thành công! Số lớp: 162


In [31]:
# Tạo Dataset và DataLoader mới
train_dataset_htr = CustomDatasetHTR(X_train_paths, y_train_encoded, transform=htr_transform)
test_dataset_htr = CustomDatasetHTR(X_test_paths, y_test_encoded, transform=htr_transform)

In [66]:
# Điều chỉnh batch_size tùy theo bộ nhớ GPU
BATCH_SIZE = 32
# NUM_WORKERS = 2 # Điều chỉnh số worker
train_loader_htr = DataLoader(train_dataset_htr, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn_htr, num_workers=0, pin_memory=True)
test_loader_htr = DataLoader(test_dataset_htr, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_htr, num_workers=0, pin_memory=True)

print(f"\nĐã tạo DataLoader với batch_size={BATCH_SIZE}")
# Kiểm tra thử một batch
# try:
#     images_batch, labels_batch, input_len_batch, label_len_batch = next(iter(train_loader_htr))
#     print("Kích thước batch ảnh:", images_batch.shape) # (N, C, H, W)
#     print("Kích thước batch label (padded):", labels_batch.shape) # (N, L_max)
#     print("Kích thước input lengths:", input_len_batch.shape) # (N,)
#     print("Kích thước label lengths:", label_len_batch.shape) # (N,)
#     print("Input lengths sample:", input_len_batch)
# except Exception as e:
#     print(f"Lỗi khi lấy batch đầu tiên từ DataLoader: {e}")


Đã tạo DataLoader với batch_size=32


In [67]:
# === KHÔNG THAY ĐỔI (Định nghĩa mô hình CRNN) ===
class CRNN(nn.Module):
    def __init__(self, img_channels, num_classes, rnn_hidden_size=256, rnn_num_layers=2, leaky_relu=False):
        super(CRNN, self).__init__()
         # Giảm số kênh CNN
        nm = [32, 64, 128, 128, 256, 256, 256]# <-- Giảm nm
        self.cnn_output_channels = nm[6] # <-- Cập nhật

        ks = [3, 3, 3, 3, 3, 3, 2]
        ps = [1, 1, 1, 1, 1, 1, 0]
        ss = [1, 1, 1, 1, 1, 1, 1]

        cnn = nn.Sequential()
        def convRelu(i, batchNormalization=False):
            nIn = img_channels if i == 0 else nm[i - 1]
            nOut = nm[i]
            cnn.add_module(f'conv{i}', nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            if batchNormalization: cnn.add_module(f'batchnorm{i}', nn.BatchNorm2d(nOut))
            if leaky_relu: cnn.add_module(f'relu{i}', nn.LeakyReLU(0.2, inplace=True))
            else: cnn.add_module(f'relu{i}', nn.ReLU(True))

        convRelu(0); cnn.add_module(f'pooling{0}', nn.MaxPool2d(2, 2)) # 64x16xW/2
        convRelu(1); cnn.add_module(f'pooling{1}', nn.MaxPool2d(2, 2)) # 128x8xW/4
        convRelu(2, True); convRelu(3); cnn.add_module(f'pooling{2}', nn.MaxPool2d((2, 2), (2, 1), (0, 1))) # 256x4x(W/4)+1 -> làm tròn W'/4
        convRelu(4, True); convRelu(5); cnn.add_module(f'pooling{3}', nn.MaxPool2d((2, 2), (2, 1), (0, 1))) # 512x2xW'/4
        convRelu(6, True) # 512x1xW'/4

        self.cnn = cnn
        self.rnn = nn.LSTM(self.cnn_output_channels, rnn_hidden_size, rnn_num_layers, bidirectional=True, batch_first=False)
        self.fc = nn.Linear(rnn_hidden_size * 2, num_classes)

    def forward(self, x):
        # CNN Part
        features = self.cnn(x) # (N, C_out, 1, W_out)
        N, C_out, H_out, W_out = features.size()
        assert H_out == 1, f"Expected height 1 after CNN, got {H_out}"
        features = features.squeeze(2) # (N, C_out, W_out)
        # Map-to-Sequence
        features = features.permute(2, 0, 1) # (W_out, N, C_out) - SeqLen = W_out
        # RNN Part
        rnn_output, _ = self.rnn(features) # (W_out, N, Hidden*2)
        # FC Part
        output = self.fc(rnn_output) # (W_out, N, NumClasses)
        return output

In [68]:
# === KHÔNG THAY ĐỔI (Khởi tạo model, loss, optimizer, checkpoint) ===
# Kiểm tra device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nSử dụng device: {device}")

model_htr = CRNN(img_channels=1, num_classes=num_classes_htr)
model_htr = model_htr.to(device)
print("Khởi tạo mô hình CRNN thành công.")

# Thử forward 1 batch để kiểm tra kích thước output
try:
    model_htr.eval()
    with torch.no_grad():
         # Sử dụng context manager để tránh lỗi dataloader nếu num_workers > 0 trên Windows
        with DataLoader(train_dataset_htr, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn_htr, num_workers=0) as temp_loader:
            test_batch = next(iter(temp_loader))
            if test_batch[0] is not None:
                images_batch_test = test_batch[0].to(device)
                output_batch_test = model_htr(images_batch_test)
                print("Kích thước output của mô hình (SeqLen, Batch, NumClasses):", output_batch_test.shape)
                # Kiểm tra SeqLen có khớp CNN_OUTPUT_WIDTH không
                if output_batch_test.shape[0] != CNN_OUTPUT_WIDTH:
                     print(f"CẢNH BÁO: Output SeqLen {output_batch_test.shape[0]} KHÔNG khớp CNN_OUTPUT_WIDTH {CNN_OUTPUT_WIDTH}. Input lengths sẽ không chính xác!")
                else:
                     print("Kiểm tra kích thước output thành công!")
            else:
                print("Lỗi: Không thể lấy batch hợp lệ để kiểm tra model.")
    model_htr.train()
except Exception as e:
    print(f"Lỗi khi thử forward model: {e}")


Sử dụng device: cpu
Khởi tạo mô hình CRNN thành công.
Lỗi khi thử forward model: __enter__


In [79]:
criterion_htr = nn.CTCLoss(blank=char_to_int[BLANK_TOKEN], reduction='mean', zero_infinity=True)
optimizer_htr = optim.AdamW(model_htr.parameters(), lr=5e-2, weight_decay=4e-5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_htr, 'min', patience=3, factor=0.5, verbose=True)

CHECKPOINT_PATH_HTR = r"G:\Compvision Final\pythonProject\checkpoint_htr_line.pth" # <--- Đổi tên checkpoint
START_EPOCH_HTR = 0
NUM_EPOCHS = 40
best_val_loss = float('inf')
train_losses = []
val_losses = []

# Load checkpoint nếu có
if os.path.exists(CHECKPOINT_PATH_HTR):
    try:
        checkpoint = torch.load(CHECKPOINT_PATH_HTR, map_location=device)
        # Load char_to_int và int_to_char từ checkpoint nếu chúng tồn tại và khớp
        # (Quan trọng nếu bạn thay đổi bộ ký tự giữa các lần chạy)
        if 'char_to_int' in checkpoint and checkpoint['char_to_int'] == char_to_int:
            print("Tải thành công char_to_int từ checkpoint.")
        else:
            print("Cảnh báo: char_to_int trong checkpoint khác hoặc không tồn tại. Sử dụng bộ ký tự hiện tại.")

        model_htr.load_state_dict(checkpoint['model_state_dict'])
        optimizer_htr.load_state_dict(checkpoint['optimizer_state_dict'])
        START_EPOCH_HTR = checkpoint.get('epoch', 0) + 1
        best_val_loss = checkpoint.get('best_val_loss', float('inf'))
        train_losses = checkpoint.get('train_losses', [])
        val_losses = checkpoint.get('val_losses', [])
        print(f"Đã tải checkpoint. Tiếp tục huấn luyện từ epoch {START_EPOCH_HTR}")
    except Exception as e:
        print(f"Lỗi khi tải checkpoint: {e}. Bắt đầu huấn luyện từ đầu.")
        START_EPOCH_HTR = 0; best_val_loss = float('inf'); train_losses = []; val_losses = []
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.")

Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.


In [73]:
# === KHÔNG THAY ĐỔI (Hàm giải mã Greedy) ===
def decode_batch(outputs, int_to_char_map, blank_token_idx):
    preds = torch.argmax(outputs.detach(), dim=2) # (SeqLen, Batch)
    preds = preds.permute(1, 0) # (Batch, SeqLen)
    decoded_texts = []
    for pred_seq in preds:
        text = ""
        last_char_idx = -1
        for idx in pred_seq:
            idx_item = idx.item()
            if idx_item == blank_token_idx or idx_item == last_char_idx:
                last_char_idx = idx_item
                continue
            text += int_to_char_map.get(idx_item, '')
            last_char_idx = idx_item
        decoded_texts.append(text)
    return decoded_texts

In [78]:
print(f"\nBắt đầu huấn luyện từ epoch {START_EPOCH_HTR + 1} đến {NUM_EPOCHS} (BỎ QUA VALIDATION)")

for epoch in range(START_EPOCH_HTR, NUM_EPOCHS):
    # --- Training Phase ---
    model_htr.train()
    running_loss = 0.0
    num_train_samples = 0
    train_progress_bar = tqdm(train_loader_htr, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]", unit="batch")

    for batch_data in train_progress_bar:
        if batch_data[0] is None: continue

        images, padded_labels, input_lengths, label_lengths = batch_data
        current_batch_size = images.size(0)
        num_train_samples += current_batch_size

        images = images.to(device)
        padded_labels = padded_labels.to(device)
        input_lengths = input_lengths.to(device)
        label_lengths = label_lengths.to(device)

        optimizer_htr.zero_grad()
        outputs = model_htr(images)
        log_probs = nn.functional.log_softmax(outputs, dim=2)

        valid_input_lengths = torch.clamp(input_lengths, max=log_probs.shape[0])
        valid_label_lengths = torch.clamp(label_lengths, min=1)
        valid_indices = (valid_label_lengths > 0) & (valid_label_lengths <= valid_input_lengths)
        if not valid_indices.any(): continue

        log_probs_valid = log_probs[:, valid_indices, :]
        padded_labels_valid = padded_labels[valid_indices]
        valid_input_lengths_filtered = valid_input_lengths[valid_indices]
        valid_label_lengths_filtered = valid_label_lengths[valid_indices]

        try:
            loss = criterion_htr(log_probs_valid, padded_labels_valid, valid_input_lengths_filtered, valid_label_lengths_filtered)
            if torch.isnan(loss) or torch.isinf(loss): continue
            loss.backward()
            optimizer_htr.step()
            running_loss += loss.item() * log_probs_valid.size(1)
            train_progress_bar.set_postfix(loss=f"{loss.item():.4f}")
        except Exception as e:
            print(f"\nLỗi trong training step: {e}")
            continue

    avg_train_loss = running_loss / num_train_samples if num_train_samples > 0 else 0.0
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} [Train] Average Loss: {avg_train_loss:.4f}")

    # --- BỎ QUA HOÀN TOÀN VALIDATION PHASE ---

    # Chỉ lưu checkpoint sau mỗi epoch (chứa model ở epoch đó)
    checkpoint_data = {
        'epoch': epoch,
        'model_state_dict': model_htr.state_dict(),
        'optimizer_state_dict': optimizer_htr.state_dict(),
        'char_to_int': char_to_int,
        'int_to_char': int_to_char,
        'train_losses': train_losses, # Chỉ còn train loss
        # Bỏ val_losses và best_val_loss
        'target_img_width': TARGET_IMG_WIDTH,
        'target_img_height': TARGET_IMG_HEIGHT,
        'cnn_output_width': CNN_OUTPUT_WIDTH
    }
    torch.save(checkpoint_data, CHECKPOINT_PATH_HTR)
    print(f"Checkpoint saved at epoch {epoch+1}") # Có thể bỏ comment nếu muốn xem log lưu

print("\nHoàn thành huấn luyện (không có validation)!")


Bắt đầu huấn luyện từ epoch 36 đến 40 (BỎ QUA VALIDATION)


Epoch 36/40 [Train]:   6%|▌         | 11/183 [00:09<02:21,  1.22batch/s, loss=0.9304]


KeyboardInterrupt: 

In [37]:
import jiwer

In [77]:
def decode_padded_labels(padded_labels, label_lengths, int_to_char_map):
    """
    Chuyển đổi batch label đã được pad và tensor độ dài tương ứng
    thành một list các chuỗi text gốc.

    Args:
        padded_labels (torch.Tensor): Tensor chứa các label đã mã hóa và pad.
                                       Shape: (Batch, MaxLength)
        label_lengths (torch.Tensor): Tensor chứa độ dài thực tế của mỗi label.
                                      Shape: (Batch,)
        int_to_char_map (dict): Từ điển ánh xạ từ index số nguyên sang ký tự.

    Returns:
        list[str]: List các chuỗi text gốc đã được giải mã.
    """
    ground_truth_texts = []
    # Chuyển tensor sang CPU nếu đang ở GPU để xử lý dễ hơn
    padded_labels_cpu = padded_labels.cpu()
    label_lengths_cpu = label_lengths.cpu()

    for i in range(padded_labels_cpu.size(0)): # Lặp qua từng sample trong batch
        label_len = label_lengths_cpu[i].item() # Lấy độ dài thực tế của label hiện tại

        # Kiểm tra độ dài hợp lệ trước khi cắt tensor
        if label_len <= 0:
            # print(f"Warning: Label length is {label_len} for sample index {i}. Decoding as empty string.")
            ground_truth_texts.append("")
            continue
        # Đảm bảo label_len không vượt quá chiều dài đã pad
        if label_len > padded_labels_cpu.shape[1]:
             # print(f"Warning: Label length {label_len} exceeds padded dimension {padded_labels_cpu.shape[1]} for sample {i}. Clamping.")
             label_len = padded_labels_cpu.shape[1] # Cắt bớt nếu độ dài không hợp lệ

        # Lấy các indices thực tế của label (bỏ phần padding)
        label_indices = padded_labels_cpu[i][:label_len].numpy() # Chuyển sang numpy array

        # Chuyển các indices thành chuỗi ký tự
        try:
            # Dùng .get(idx, '') để tránh lỗi nếu gặp index không có trong map
            text = "".join([int_to_char_map.get(idx, '') for idx in label_indices])
            ground_truth_texts.append(text)
        except Exception as e:
            print(f"Error decoding indices for sample {i}: {label_indices}. Error: {e}")
            ground_truth_texts.append("[DECODING ERROR]") # Hoặc trả về chuỗi rỗng

    return ground_truth_texts

In [76]:
# ... (Sau khi vòng lặp for epoch... huấn luyện kết thúc) ...

print("\n--- Bắt đầu đánh giá trên tập Test sau khi huấn luyện ---")

# 1. Load Model từ Checkpoint cuối cùng
final_checkpoint_path = CHECKPOINT_PATH_HTR # Đường dẫn checkpoint cuối cùng bạn đã lưu
load_path = final_checkpoint_path

if os.path.exists(load_path):
    print(f"Đang tải model từ checkpoint cuối cùng: {load_path}")
    try:
        checkpoint = torch.load(load_path, map_location=device)

        if 'model_state_dict' in checkpoint:
            print("Phát hiện file checkpoint đầy đủ.")
            state_dict_to_load = checkpoint['model_state_dict']
            # Lấy lại char maps từ checkpoint là tốt nhất
            if 'char_to_int' in checkpoint:
                char_to_int = checkpoint['char_to_int']
                print("Đã tải char_to_int từ checkpoint.")
            else:
                print("Cảnh báo: Không tìm thấy char_to_int trong checkpoint. Sử dụng map hiện tại.")
            if 'int_to_char' in checkpoint:
                int_to_char = checkpoint['int_to_char']
                print("Đã tải int_to_char từ checkpoint.")
            else:
                 print("Cảnh báo: Không tìm thấy int_to_char trong checkpoint. Sử dụng map hiện tại.")

            num_classes_loaded = len(char_to_int)
            print(f"Số lớp được tải từ checkpoint: {num_classes_loaded}")

            # Lấy BLANK_TOKEN_INDEX từ char_to_int đã load
            if BLANK_TOKEN in char_to_int:
                BLANK_TOKEN_INDEX = char_to_int[BLANK_TOKEN]
            else:
                print(f"Cảnh báo: '{BLANK_TOKEN}' không có trong char_to_int đã load. Dùng index 0.")
                BLANK_TOKEN_INDEX = 0

        else:
            print(f"Lỗi: File {load_path} không chứa 'model_state_dict'.")
            exit()

        # Khởi tạo cấu trúc model với số lớp đúng
        model_to_evaluate = CRNN(img_channels=1, num_classes=num_classes_loaded).to(device)
        model_to_evaluate.load_state_dict(state_dict_to_load)
        print("Tải trọng số model để đánh giá thành công.")
        model_to_evaluate.eval() # Chuyển sang chế độ đánh giá

        # 2. Chuẩn bị DataLoader cho tập Test
        if 'test_loader_htr' not in locals() or test_loader_htr is None:
             print("Lỗi: test_loader_htr chưa được tạo hoặc không hợp lệ.")
             exit()

        # 3. Chạy vòng lặp đánh giá
        total_wer_info_final = {'hits': 0, 'substitutions': 0, 'deletions': 0, 'insertions': 0, 'words': 0}
        total_cer_info_final = {'hits': 0, 'substitutions': 0, 'deletions': 0, 'insertions': 0, 'chars': 0}
        evaluation_progress_bar = tqdm(test_loader_htr, desc="Evaluating Test Set", unit="batch")

        with torch.no_grad():
            for batch_data in evaluation_progress_bar:
                if batch_data[0] is None: continue

                images, padded_labels, input_lengths, label_lengths = batch_data
                images = images.to(device)
                padded_labels = padded_labels.to(device)
                label_lengths = label_lengths.to(device)

                # 4. Dự đoán và Giải mã
                outputs = model_to_evaluate(images)
                predicted_texts = decode_batch(outputs.cpu(), int_to_char, BLANK_TOKEN_INDEX)

                # 5. Lấy nhãn gốc
                ground_truth_texts = decode_padded_labels(padded_labels, label_lengths, int_to_char)

                # 6. Tính toán và Tích lũy
                try:
                    # --- Tính WER (Giữ nguyên) ---
                    wer_measures = jiwer.compute_measures(ground_truth_texts, predicted_texts)
                    total_wer_info_final['substitutions'] += wer_measures['substitutions']
                    total_wer_info_final['deletions'] += wer_measures['deletions']
                    total_wer_info_final['insertions'] += wer_measures['insertions']
                    total_wer_info_final['hits'] += wer_measures['hits']
                    total_wer_info_final['words'] += sum(len(t.split()) for t in ground_truth_texts if t)

                    # --- Tính CER bằng Workaround ---
                    # Biến đổi chuỗi: thêm khoảng trắng giữa các ký tự
                    gt_chars = [" ".join(list(t)) if t else "" for t in ground_truth_texts]
                    pred_chars = [" ".join(list(p)) if p else "" for p in predicted_texts]

                    # Gọi compute_measures trên chuỗi đã biến đổi (KHÔNG dùng char_level=True)
                    # jiwer sẽ coi mỗi ký tự là một "từ"
                    cer_measures_workaround = jiwer.compute_measures(gt_chars, pred_chars)

                    # Cộng dồn lỗi ký tự từ kết quả workaround
                    total_cer_info_final['substitutions'] += cer_measures_workaround['substitutions']
                    total_cer_info_final['deletions'] += cer_measures_workaround['deletions']
                    total_cer_info_final['insertions'] += cer_measures_workaround['insertions']
                    total_cer_info_final['hits'] += cer_measures_workaround['hits']

                    # Cộng dồn TỔNG SỐ KÝ TỰ GỐC (quan trọng!)
                    total_cer_info_final['chars'] += sum(len(t) for t in ground_truth_texts if t)
                    # -----------------------------

                except Exception as e:
                    # Thay đổi thông báo lỗi để rõ ràng hơn
                    print(f"\nLỗi khi tính WER/CER (workaround) với jiwer: {e}")


        # 7. Tính WER/CER cuối cùng
        final_word_errors = total_wer_info_final['substitutions'] + total_wer_info_final['deletions'] + total_wer_info_final['insertions']
        final_wer = final_word_errors / total_wer_info_final['words'] if total_wer_info_final['words'] > 0 else float('inf')

        # Tính CER từ các thành phần lỗi đã cộng dồn (theo workaround)
        final_char_errors = total_cer_info_final['substitutions'] + total_cer_info_final['deletions'] + total_cer_info_final['insertions']
        # Tính CER dựa trên tổng số ký tự gốc đã đếm
        final_cer = final_char_errors / total_cer_info_final['chars'] if total_cer_info_final['chars'] > 0 else float('inf')

        print("\n--- Kết quả đánh giá cuối cùng trên tập Test ---")
        print(f"Word Error Rate (WER): {final_wer:.4f}")
        print(f"Character Error Rate (CER): {final_cer:.4f}") # CER sẽ không còn là inf nếu tính đúng
        print(f"(Dựa trên {total_wer_info_final['words']} từ và {total_cer_info_final['chars']} ký tự gốc)")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file checkpoint tại {load_path} để đánh giá.")
    except KeyError as e:
         print(f"Lỗi: Key không tìm thấy trong checkpoint khi tải ({e}). File checkpoint có thể bị lỗi hoặc không đúng định dạng.")
    except Exception as e:
        print(f"Lỗi không xác định trong quá trình đánh giá cuối cùng: {e}")
else:
    print(f"Không tìm thấy checkpoint tại {load_path} để thực hiện đánh giá cuối cùng.")

print("\nScript HTR hoàn tất.")


--- Bắt đầu đánh giá trên tập Test sau khi huấn luyện ---
Đang tải model từ checkpoint cuối cùng: G:\Compvision Final\pythonProject\checkpoint_htr_line.pth
Phát hiện file checkpoint đầy đủ.
Đã tải char_to_int từ checkpoint.
Đã tải int_to_char từ checkpoint.
Số lớp được tải từ checkpoint: 162
Tải trọng số model để đánh giá thành công.


Evaluating Test Set: 100%|██████████| 46/46 [00:05<00:00,  8.46batch/s]


--- Kết quả đánh giá cuối cùng trên tập Test ---
Word Error Rate (WER): 0.9977
Character Error Rate (CER): 0.6938
(Dựa trên 22737 từ và 98048 ký tự gốc)

Script HTR hoàn tất.
